In [1]:
# Import Libraries
from collections import OrderedDict
import pandas as pd
import numpy as np
import pylogit as pl

In [2]:
# Import Data
data_wide=pd.read_csv('LRdata2.csv')
data_wide.tail()

,respond_id,TOD,food_stamps,unemployment,poverty,costburden_home,costburden_rent,overcrowding,log_rent,median_rent,...,houseprice_index,Black,Hispanic,Asian,child,senior,female,singlehousehold,veteran,nobachdegree
5,6,2,3.61,-10.7,-0.6,-5.43494,7.55321,0.000000,0.096560,247,...,14.3,-5.1,-3.9,-8.2,1.1,2.8,-0.2,-2.3,0.8,-16.1
6,7,2,-3.00,-5.7,-11.3,-3.21204,-2.82286,2.014053,0.119807,453,...,-9.9,-5.7,2.5,1.2,-0.9,2.5,-3.2,-2.8,0.8,2.4
7,8,2,-2.30,-2.3,-4.4,-4.94328,0.29547,-1.171336,0.045056,149,...,-9.9,-5.1,10.8,1.5,-7.1,1.2,-2.6,-2.7,-0.8,-25.6
8,9,2,-3.78,0.2,0.0,-10.16552,7.89414,-0.890264,0.065778,200,...,8.1,1.5,-3.7,10.4,2.7,-0.9,-3.0,-5.2,0.3,0.0
9,10,2,2.90,3.0,6.3,1.39682,-18.18025,0.603994,0.046762,167,...,-0.2,-1.4,4.6,-4.1,-0.5,6.7,-0.8,-1.4,-3.2,8.9


In [3]:
data_wide.columns
data_wide["YesTOD"] = 1
data_wide["NoTOD"] = 1
data_wide

,respond_id,TOD,food_stamps,unemployment,poverty,costburden_home,costburden_rent,overcrowding,log_rent,median_rent,...,Hispanic,Asian,child,senior,female,singlehousehold,veteran,nobachdegree,YesTOD,NoTOD
0,1,1,0.62,-1.9,-2.4,-1.051769,3.837397,1.035673,-0.004815,-10,...,0.1,-3.7,-2.6,-0.1,-4.1,0.7,-1.7,-19.6,1,1
1,2,1,5.20,-1.8,-0.6,7.018410,5.834024,6.172411,0.076827,340,...,5.7,1.3,1.8,1.1,-1.7,-3.4,-4.4,1.8,1,1
2,3,1,-1.10,-3.2,-2.6,-7.398843,-2.817393,-1.581028,0.099307,325,...,-2.8,5.5,-4.6,5.7,0.3,-7.5,-0.3,-5.3,1,1
3,4,1,2.76,-5.6,2.7,-25.374703,-3.413288,1.105962,0.035779,169,...,-5.4,10.1,3.8,-1.7,1.2,-3.6,-5.5,-0.5,1,1
4,5,1,-0.70,-6.8,-2.6,-10.738466,-4.345783,-0.128306,0.116734,379,...,-6.4,10.0,-1.5,-1.8,2.5,2.8,-1.0,-4.0,1,1
5,6,2,3.61,-10.7,-0.6,-5.434940,7.553210,0.000000,0.096560,247,...,-3.9,-8.2,1.1,2.8,-0.2,-2.3,0.8,-16.1,1,1
6,7,2,-3.00,-5.7,-11.3,-3.212040,-2.822860,2.014053,0.119807,453,...,2.5,1.2,-0.9,2.5,-3.2,-2.8,0.8,2.4,1,1
7,8,2,-2.30,-2.3,-4.4,-4.943280,0.295470,-1.171336,0.045056,149,...,10.8,1.5,-7.1,1.2,-2.6,-2.7,-0.8,-25.6,1,1
8,9,2,-3.78,0.2,0.0,-10.165520,7.894140,-0.890264,0.065778,200,...,-3.7,10.4,2.7,-0.9,-3.0,-5.2,0.3,0.0,1,1
9,10,2,2.90,3.0,6.3,1.396820,-18.180250,0.603994,0.046762,167,...,4.6,-4.1,-0.5,6.7,-0.8,-1.4,-3.2,8.9,1,1


In [4]:
ind_variables = data_wide.columns.tolist()[2:24]
print(ind_variables)

['food_stamps', 'unemployment', 'poverty', 'costburden_home', 'costburden_rent', 'overcrowding', 'log_rent', 'median_rent', 'rent_vacancy', 'renter_occ', 'med_contract_rent', 'med_rent_utility', 'houseprice_index', 'Black', 'Hispanic', 'Asian', 'child', 'senior', 'female', 'singlehousehold', 'veteran', 'nobachdegree']


In [5]:
alt_varying_variables = {}

# 
availability_variables = {1: 'YesTOD',
                         2: 'NoTOD'}

# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "todlocation"


# Create a custom id column that ignores the fact that this is a 
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "respond_id"
# data_wide[obs_id_column] = np.arange(data_wide.shape[0],dtype=int) + 1


# Create a variable recording the choice column
choice_column = 'TOD'



In [6]:
# Perform the conversion to long-format
data_long = pl.convert_wide_to_long(data_wide, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables,
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
data_long

,respond_id,todlocation,TOD,food_stamps,unemployment,poverty,costburden_home,costburden_rent,overcrowding,log_rent,...,houseprice_index,Black,Hispanic,Asian,child,senior,female,singlehousehold,veteran,nobachdegree
0,1,1,1,0.62,-1.9,-2.4,-1.051769,3.837397,1.035673,-0.004815,...,14.3,-8.6,0.1,-3.7,-2.6,-0.1,-4.1,0.7,-1.7,-19.6
1,1,2,0,0.62,-1.9,-2.4,-1.051769,3.837397,1.035673,-0.004815,...,14.3,-8.6,0.1,-3.7,-2.6,-0.1,-4.1,0.7,-1.7,-19.6
2,2,1,1,5.20,-1.8,-0.6,7.018410,5.834024,6.172411,0.076827,...,-9.9,-2.4,5.7,1.3,1.8,1.1,-1.7,-3.4,-4.4,1.8
3,2,2,0,5.20,-1.8,-0.6,7.018410,5.834024,6.172411,0.076827,...,-9.9,-2.4,5.7,1.3,1.8,1.1,-1.7,-3.4,-4.4,1.8
4,3,1,1,-1.10,-3.2,-2.6,-7.398843,-2.817393,-1.581028,0.099307,...,-9.9,-3.2,-2.8,5.5,-4.6,5.7,0.3,-7.5,-0.3,-5.3
5,3,2,0,-1.10,-3.2,-2.6,-7.398843,-2.817393,-1.581028,0.099307,...,-9.9,-3.2,-2.8,5.5,-4.6,5.7,0.3,-7.5,-0.3,-5.3
6,4,1,1,2.76,-5.6,2.7,-25.374703,-3.413288,1.105962,0.035779,...,8.1,1.9,-5.4,10.1,3.8,-1.7,1.2,-3.6,-5.5,-0.5
7,4,2,0,2.76,-5.6,2.7,-25.374703,-3.413288,1.105962,0.035779,...,8.1,1.9,-5.4,10.1,3.8,-1.7,1.2,-3.6,-5.5,-0.5
8,5,1,1,-0.70,-6.8,-2.6,-10.738466,-4.345783,-0.128306,0.116734,...,-0.2,-10.7,-6.4,10.0,-1.5,-1.8,2.5,2.8,-1.0,-4.0
9,5,2,0,-0.70,-6.8,-2.6,-10.738466,-4.345783,-0.128306,0.116734,...,-0.2,-10.7,-6.4,10.0,-1.5,-1.8,2.5,2.8,-1.0,-4.0


In [7]:
basic_specification = OrderedDict()
basic_names = OrderedDict()


basic_specification["intercept"] = [1]
basic_names["intercept"] = ['ASC']

basic_specification["food_stamps"] = [1]
basic_names["food_stamps"] = ['Food Stamps']

basic_specification["unemployment"] = [1]
basic_names["unemployment"] = ['Unemployment']

basic_specification["poverty"] = [1]
basic_names["poverty"] = ['Poverty Rate']

basic_specification["costburden_home"] = [1]
basic_names["costburden_home"] = ['Cost Burdened Homeowners']

basic_specification["costburden_rent"] = [1]
basic_names["costburden_rent"] = ['Cost Burdened Renters']

basic_specification["overcrowding"] = [1]
basic_names["overcrowding"] = ['Overcrowding']

basic_specification["log_rent"] = [1]
basic_names["log_rent"] = ['log rent']

# basic_specification["median_rent"] = [1]
# basic_names["median_rent"] = ['Median rent']

basic_specification["rent_vacancy"] = [1]
basic_names["rent_vacancy"] = ['Percent Rental Vacancy']

basic_specification["renter_occ"] = [1]
basic_names["renter_occ"] = ['Rental Occupied Units']

# basic_specification["med_contract_rent"] = [1]
# basic_names["med_contract_rent"] = ['Median Contract Rent']

basic_specification["houseprice_index"] = [1]
basic_names["houseprice_index"] = ['Housing Price Index']

basic_specification["Black"] = [1]
basic_names["Black"] = ['Percent Black']

basic_specification["Hispanic"] = [1]
basic_names["Hispanic"] = ['Percent Hispanic/Latino']

basic_specification["Asian"] = [1]
basic_names["Asian"] = ['Percent Asian']

basic_specification["child"] = [1]
basic_names["child"] = ['Percent Child']

basic_specification["senior"] = [1]
basic_names["senior"] = ['Percent Senior']

basic_specification["female"] = [1]
basic_names["female"] = ['Percent Female']

basic_specification["singlehousehold"] = [1]
basic_names["singlehousehold"] = ['Singe Household']

basic_specification['veteran'] = [1]
basic_names['veteran'] = ['Percent veteran']

basic_specification['nobachdegree'] = [1]
basic_names['nobachdegree'] = ['Percent without a Bachelors Degree']

In [8]:
# Estimate the multinomial logit model (MNL)
trial_mnl = pl.create_choice_model(data=data_long,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

# Specify the initial values and method for the optimization.
trial_mnl.fit_mle(np.zeros(20))

# Look at the estimation results
trial_mnl.get_statsmodels_summary()

Log-likelihood at zero: -6.9315
Initial Log-likelihood: -6.9315
Estimation Time for Point Estimation: 0.05 seconds.
Final log-likelihood: -0.0000


C:\Users\natal\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
C:\Users\natal\anaconda3\lib\site-packages\pylogit\base_multinomial_cm_v2.py:1228: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.cov)),
C:\Users\natal\anaconda3\lib\site-packages\pylogit\base_multinomial_cm_v2.py:1259: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.robust_cov)),


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         TOD   No. Observations:                   10
Model:             Multinomial Logit Model   Df Residuals:                      -10
Method:                                MLE   Df Model:                           20
Date:                     Sat, 06 May 2023   Pseudo R-squ.:                   1.000
Time:                             18:41:26   Pseudo R-bar-squ.:              -1.885
AIC:                                40.000   Log-Likelihood:                 -0.000
BIC:                                46.052   LL-Null:                        -6.931
======================================================================================================
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
ASC                                   -0.4427        nan        nan        nan         nan         nan
Food Stamps                            2.1358        nan        nan        nan         nan         nan
Unemployment                           1.9920      3e+07   6.63e-08      1.000   -5.89e+07    5.89e+07
Poverty Rate                           0.6009        nan        nan        nan         nan         nan
Cost Burdened Homeowners              -0.8803   1.31e+07  -6.71e-08      1.000   -2.57e+07    2.57e+07
Cost Burdened Renters                 -1.3794   1.75e+07  -7.88e-08      1.000   -3.43e+07    3.43e+07
Overcrowding                           0.6047   1.49e+07   4.06e-08      1.000   -2.92e+07    2.92e+07
log rent                              -0.0786        nan        nan        nan         nan         nan
Percent Rental Vacancy                -2.8817   3.48e+07  -8.28e-08      1.000   -6.82e+07    6.82e+07
Rental Occupied Units                  5.2584   3.46e+07   1.52e-07      1.000   -6.78e+07    6.78e+07
Housing Price Index                   -2.5642        nan        nan        nan         nan         nan
Percent Black                          0.6162        nan        nan        nan         nan         nan
Percent Hispanic/Latino               -3.0182   3.87e+07   -7.8e-08      1.000   -7.59e+07    7.59e+07
Percent Asian                          0.9418        nan        nan        nan         nan         nan
Percent Child                         -2.2857        nan        nan        nan         nan         nan
Percent Senior                         0.0319   3.33e+07   9.56e-10      1.000   -6.53e+07    6.53e+07
Percent Female                         1.2241        nan        nan        nan         nan         nan
Singe Household                       -0.8071        nan        nan        nan         nan         nan
Percent veteran                       -2.5266   1.68e+07  -1.51e-07      1.000   -3.29e+07    3.29e+07
Percent without a Bachelors Degree    -1.7363        nan        nan        nan         nan         nan
======================================================================================================
"""